# UR5 Kinematics simulation

We will use th "ur5_e_moveit_config" package in "src/fmauch_universal_robot" folder for simulation purposes. 

Detailed information will be found in readme file on: https://github.com/fmauch/universal_robot/tree/melodic-devel-staging

We will be able to simulate the Forward and Inverse Kinematics

## Forward Kinematics
Be sure you have installed "rqt-joint-trajectory-controller" package:

In [ ]:
sudo apt-get install ros-melodic-rqt-joint-trajectory-controller

### Usage with Gazebo Simulation

First of all, we run the simulated ur5 model with the following command:

In [ ]:
roslaunch ur_e_gazebo ur5e.launch

To move each one of the joints we will use the package installed before. 

In [ ]:
rosrun rqt_joint_trajectory_controller rqt_joint_trajectory_controller

On the window controller, we choose 'arm_controller'. After that, all the possible joints of UR5 will appear. Changing these values we will observe the movement in Gazebo. 

<img src="./Images/2_gazebo_joint_controller.png">

## Inverse Kinematics
For inverse kinematics you can use moveIt!
### MoveIt! with a simulated robot

Again, you can use MoveIt! to control the simulated robot.

Tutorial
http://docs.ros.org/en/melodic/api/moveit_tutorials/html/index.html

Videos
https://www.youtube.com/watch?v=b4T577d39dE

https://www.youtube.com/watch?v=BxCik8OI1Fw

As MoveIt! seems to have difficulties with finding plans for the UR with full joint limits [-2pi, 2pi], there is a joint_limited version using joint limits restricted to [-pi,pi]. In order to use this joint limited version, simply use the launch file arguments 'limited', i.e.:

In [ ]:
roslaunch ur_e_gazebo ur5e.launch limited:=true

For setting up the MoveIt! nodes to allow motion planning run:

In [ ]:
roslaunch ur5_e_moveit_config ur5_e_moveit_planning_execution.launch sim:=true limited:=true

For starting up RViz with a configuration including the MoveIt! Motion Planning plugin run:

In [ ]:
roslaunch ur5_e_moveit_config moveit_rviz.launch config:=true

<img src="./Images/2_gazebo_rviz_IK.png">

You can:
- Plan: with Motion Planning you can view the results in rviz
- execute: the planned movement is executed in gazebo

### Specific package for Kinematics control

We can create a specific package for kinematics control.

Create the ur5control package with dependencies:
- moveit_core 
- moveit_visual_tools 
- moveit_ros_planning_interface 
- interactive_markers 
- tf2_geometry_msgs

In [ ]:
catkin_create_pkg ur5control rospy std_msgs moveit_core moveit_visual_tools moveit_ros_planning_interface interactive_markers tf2_geometry_msgs

create a specific python file to control the UR5 arm movement in:
- Forward Kinematics --> Joints control
- Inverse Kinematics --> Pose Control

Run first the gazebo model, planning execution and moveit rviz

In [ ]:
roslaunch ur_e_gazebo ur5e.launch limited:=true
roslaunch ur5_e_moveit_config ur5_e_moveit_planning_execution.launch sim:=true limited:=true
roslaunch ur5_e_moveit_config moveit_rviz.launch config:=true

### Joints control
We create the "ur5e_joints_control1.py" to control the different link angles:



In [ ]:
rosrun ur5control ur5e_joints_control1.py 

<img src="./Images/2_control_joints1.png">

Running this python file, we will observe the current joint values. Let's explain a little bit the code of this file.  

In [ ]:
#! /usr/bin/env python

import math
from tf.transformations import euler_from_quaternion, quaternion_from_euler
import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg

#initialize ros node for the process, using the tutorial of move group
moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node('move_group_python_interface_tutorial', anonymous=True)

#defining the robot and scene for collision checking
robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()   
#defining the group that we will control, calling it by the name that we stipulate before    
group = moveit_commander.MoveGroupCommander("manipulator")
display_trajectory_publisher = rospy.Publisher('/move_group/display_planned_path', moveit_msgs.msg.DisplayTrajectory, queue_size=1)

#On the group called, there are the six joints of the robot, adn we called the values of the joints planned
group_variable_values = group.get_current_joint_values()

#We divide the joint values on 6 (according to the number of joints), which their corresponding orentation
group_variable_values[0] = math.radians(0)
group_variable_values[1] = math.radians(-90)
group_variable_values[2] = math.radians(-90)
group_variable_values[3] = math.radians(0)
group_variable_values[4] = math.radians(0)
group_variable_values[5] = math.radians(0)

#Specify a target joint configuration for the group
group.set_joint_value_target(group_variable_values)

plan2 = group.plan()

rospy.sleep(5)
group.go(wait=True)
rospy.sleep(5)

#Here we print the current joint values
print ("Current Joint Values:")
joint_target2=group.get_current_joint_values()
print ("q1 = "+str(round(math.degrees(joint_target2[0]),0)))
print ("q2 = "+str(round(math.degrees(joint_target2[1]),0)))
print ("q3 = "+str(round(math.degrees(joint_target2[2]),0)))
print ("q4 = "+str(round(math.degrees(joint_target2[3]),0)))
print ("q5 = "+str(round(math.degrees(joint_target2[4]),0)))
print ("q6 = "+str(round(math.degrees(joint_target2[5]),0)))
#Obtaining current pose of the end-effector of the group on the 3 axis
print ("Current Pose:")
pose_target2=group.get_current_pose()
qx=round(pose_target2.pose.orientation.x,2)
qy=round(pose_target2.pose.orientation.y,2)
qz=round(pose_target2.pose.orientation.z,2)
qw=round(pose_target2.pose.orientation.w,2)
print ("x = "+str(round(pose_target2.pose.position.x,1)))
print ("y = "+str(round(pose_target2.pose.position.y,1)))
print ("z = "+str(round(pose_target2.pose.position.z,1)))
print ("qx = "+str(qx))
print ("qy = "+str(qy))
print ("qz = "+str(qz))
print ("qw = "+str(qw))
#Calculate the aircraft principal axis
quat2=[qx, qy, qz, qw]
(roll, pitch, yaw)=euler_from_quaternion(quat2)
print ("roll = "+str(math.degrees(roll)))
print ("pitch = "+str(math.degrees(pitch)))
print ("yaw = "+str(math.degrees(yaw)))

moveit_commander.roscpp_shutdown()

### Pose control

We create the "ur5e_pose_control1.py" to control the pose target:

In [ ]:
rosrun ur5control ur5e_pose_control1.py 

<img src="./Images/2_control_pose1.png">

On this case, we are going to do the same, comment the code that shows the position of the target.

In [ ]:
#! /usr/bin/env python

import math
from tf.transformations import euler_from_quaternion, quaternion_from_euler
import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg

#initialize ros node for the process, using the tutorial of move group
moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node('move_group_python_interface_tutorial', anonymous=True)

#defining the robot and scene for collision checking
robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface() 
#defining the group that we will control, calling it by the name that we stipulate before     
group = moveit_commander.MoveGroupCommander("manipulator")
display_trajectory_publisher = rospy.Publisher('/move_group/display_planned_path', moveit_msgs.msg.DisplayTrajectory, queue_size=1)

#Representation of pose and orientation
pose_target = geometry_msgs.msg.Pose()
#Defining orientation and position
pose_target.position.x = -0.3
pose_target.position.y = 0.5
pose_target.position.z = 0.5
r=math.radians(0)
p=math.radians(0)
w=math.radians(0)
quat1=quaternion_from_euler(r,p,w)

pose_target.orientation.x = quat1[0]
pose_target.orientation.y = quat1[1]
pose_target.orientation.z = quat1[2]
pose_target.orientation.w = quat1[3]
group.set_pose_target(pose_target)

plan1 = group.plan()

rospy.sleep(1)
group.go(wait=True)
rospy.sleep(1)

#Here we print the current joint values
print ("Current Joint Values:")
joint_target2=group.get_current_joint_values()
print ("q1 = "+str(round(math.degrees(joint_target2[0]),0)))
print ("q2 = "+str(round(math.degrees(joint_target2[1]),0)))
print ("q3 = "+str(round(math.degrees(joint_target2[2]),0)))
print ("q4 = "+str(round(math.degrees(joint_target2[3]),0)))
print ("q5 = "+str(round(math.degrees(joint_target2[4]),0)))
print ("q6 = "+str(round(math.degrees(joint_target2[5]),0)))
#Obtaining current pose of the end-effector of the group on the 3 axis
print ("Current Pose:")
pose_target2=group.get_current_pose()
qx=round(pose_target2.pose.orientation.x,2)
qy=round(pose_target2.pose.orientation.y,2)
qz=round(pose_target2.pose.orientation.z,2)
qw=round(pose_target2.pose.orientation.w,2)
print ("x = "+str(round(pose_target2.pose.position.x,1)))
print ("y = "+str(round(pose_target2.pose.position.y,1)))
print ("z = "+str(round(pose_target2.pose.position.z,1)))
print ("qx = "+str(qx))
print ("qy = "+str(qy))
print ("qz = "+str(qz))
print ("qw = "+str(qw))
#Calculate the aircraft principal axis
quat2=[qx, qy, qz, qw]
(roll, pitch, yaw)=euler_from_quaternion(quat2)
print ("roll = "+str(math.degrees(roll)))
print ("pitch = "+str(math.degrees(pitch)))
print ("yaw = "+str(math.degrees(yaw)))

moveit_commander.roscpp_shutdown()

### External Kinematics control

If you want to control the ur5e kinematics using an external device, you will need to communicate with it according to:

<img src="./Images/2_pen_control_node.png">

We will create 2 python files:
- "ur5e_pose2robot.py" to define the /ur5e_node
- "ur5e_pen_control.py" to define the /pen_node

We can create a "ur5e_bringup_sim.launch" launch file to start the robot arm in gazebo and the planning execution

In [ ]:
<launch>
  <arg name="sim" default="true" />
  <arg name="limited" default="true"/>
  <!-- Launch ur5e -->
   <include file="$(find ur_e_gazebo)/launch/ur5e.launch">
    <arg name="limited" default="$(arg limited)"/>
  </include>
  <!-- Launch planning execution -->
  <include file="$(find ur5_e_moveit_config)/launch/ur5_e_moveit_planning_execution.launch">
    <arg name="limited" default="$(arg limited)"/>
    <arg name="sim" default="$(arg sim)"/>
  </include>
</launch>

we have created a "ur5e_pose_control.launch" to start the 2 nodes to control the pose with a pen device

In [ ]:
<launch>
  <node name="pen_control" pkg="ur5control" type="ur5e_pen_control.py" output="screen">
  </node>
  <node name="ur5e_node" pkg="ur5control" type="ur5e_pose2robot.py" output="screen">
  </node>
</launch>

We can start all the nodes:

In [ ]:
roslaunch ur5control ur5e_bringup_sim.launch
roslaunch ur5control ur5e_pose_control.launch

<img src="./Images/1_bringup1.png">